## Cross Attention


 Cross-Attention is a fundamental mechanism in modern AI, especially in Tranformer models. It allows a neural network to dynamically relate and fuse information from different input sequences.

 